In [1]:
DEPTH = 3066
TARGET_X = 13
TARGET_Y = 726

def erosion(gi):
    return (DEPTH + gi) % 20183

gi = [[None] * (TARGET_Y + 1) for _ in range(TARGET_X + 1)]
el = [[None] * (TARGET_Y + 1) for _ in range(TARGET_X + 1)]
for x in range(TARGET_X + 1):
    g = gi[x][0] = x * 16807
    el[x][0] = erosion(g)
for y in range(1, TARGET_Y + 1):
    g = gi[0][y] = y * 48271 
    el[0][y] = erosion(g)
for x in range(1, TARGET_X + 1):
    for y in range(1, TARGET_Y + 1):
        g = gi[x][y] = el[x-1][y] * el[x][y-1]
        el[x][y] = erosion(g)
gi[TARGET_X][TARGET_Y] = 0
el[TARGET_X][TARGET_Y] = erosion(0)



In [2]:
sum(
    sum(
        el[x][y] % 3
        for y in range(TARGET_Y + 1)
    )
    for x in range(TARGET_X + 1)
)

10115

In [3]:
from heapq import heappush, heappop

DEPTH = 3066
TARGET_X = 13
TARGET_Y = 726

# DEPTH = 510
# TARGET_X = 10
# TARGET_Y = 10

def erosion(gi):
    return (DEPTH + gi) % 20183

ROCKY = NEITHER = 0
WET = TORCH = 1
NARROW = CLIMB = 2

cave = {
    (0,0): (0, erosion(0), erosion(0) % 3),
    (TARGET_X, TARGET_Y): (0, erosion(0), erosion(0) % 3),
}

seen = set()
init_state = (0, 0, TORCH)
target_state = (TARGET_X, TARGET_Y, TORCH)
seen.add(init_state)
hq = [(0, init_state)]

reequip = {
    (ROCKY, TORCH): CLIMB,
    (ROCKY, CLIMB): TORCH,
    (WET, NEITHER): CLIMB,
    (WET, CLIMB): NEITHER,
    (NARROW, TORCH): NEITHER,
    (NARROW, NEITHER): TORCH,
}

def get_cave(x, y):
    if (x, y) not in cave:
        if y == 0:
            gi = x * 16807
        elif x == 0:
            gi = y * 48271
        else:
            gi = get_cave(x-1, y)[1] * get_cave(x, y-1)[1]
        el = erosion(gi)        
        cave[(x, y)] = (gi, el, el % 3)
    return cave[(x, y)]

while hq[0][1] != target_state:
    t, (x, y, tool) = heappop(hq)
    for x1, y1 in [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]:
        if x1 < 0 or y1 < 0:
            continue
        new_state = (x1, y1, tool)
        if new_state in seen:
            continue
        if tool == get_cave(x1, y1)[2]:
            # can't go there with current tool
            continue
        seen.add(new_state)
        heappush(hq, (t+1, new_state))
    new_tool = reequip[get_cave(x, y)[2], tool]
    new_state = (x, y, new_tool)
    if new_state not in seen:
        seen.add(new_tool)
        heappush(hq, (t+7, new_state))
    
hq[0][0]

990